<a href="https://colab.research.google.com/github/alti-tude/video-slide-matching/blob/master/tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Run This Once to Initialize the Drive

#### Right click and select save to My Drive

https://drive.google.com/open?id=1J84AcNchprI0DndeaTVFSUZWF2lMo5uu

In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir '/content/gdrive/My Drive/data'
!cp -r '/content/gdrive/My Drive/Data' '/content/gdrive/My Drive/data'

# Test Generator

In [2]:
import os
root = '/content/gdrive/My Drive/data/Data/Dataset/'
testroot = '/'.join(root.split('/')[:-2]) + '/new_test/'

data = []
data_ind = []
for curdir in os.listdir(root):
    frames = []
    
    for imgs in os.listdir(root+curdir):
        if 'ppt' not in imgs:
            frames.append(root+curdir+'/'+imgs)
            data_ind.append({'ppt': root+curdir+'/ppt.jpg', 'frame':root+curdir+'/'+imgs})
            
    data.append({'ppt': root+curdir+'/ppt.jpg', 'frames':frames})
    
print(data_ind[1])

{'ppt': '/content/gdrive/My Drive/data/Data/Dataset/03_7/ppt.jpg', 'frame': '/content/gdrive/My Drive/data/Data/Dataset/03_7/0.jpg'}


In [0]:
def cp(source, dest):
    ip = open(source, 'rb')
    data = b''
    while True:
        chunk = ip.read()
        if len(chunk) == 0:
            break
        data = data + chunk
    
    ip.close()
    
    op = open(dest, 'wb')
    op.write(data)

In [0]:
def write_sol(l, testroot):
    with open(testroot+'sol.txt', 'w') as sol: 
        for i in range(len(l)):
            sol.write("%s.jpg %s.jpg\n" %(i, l[i]['ppt'].split('/')[-2]))

In [0]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



###  Rerun this for a new test

In [0]:
import random

testset_size = random.randint(5, 150)
testset = random.sample(data_ind, testset_size)


In [0]:
import os
print(testroot)
try:
    os.mkdir(testroot)
    os.mkdir(testroot+'frames/')
    os.mkdir(testroot+'ppts/')
except Exception as e:
    print(e)
    for i in os.listdir(testroot+'frames/'):
        os.remove(testroot+'frames/'+i)
    for i in os.listdir(testroot+'ppts/'):
        os.remove(testroot+'ppts/'+i)
        
out = display(progress(0, testset_size), display_id=True)

for i in range(len(testset)):
    frame = testset[i]['frame']
    ppt = testset[i]['ppt']
    ppt_name = ppt.split('/')[-2]
    
    cp(ppt, testroot+'ppts/'+ppt_name+'.jpg')
    cp(frame, testroot+'frames/'+str(i)+'.jpg')
    out.update(progress(i+1, testset_size))
#     print(frame)

write_sol(testset, testroot)

/content/gdrive/My Drive/data/Data/new_test/
[Errno 17] File exists: '/content/gdrive/My Drive/data/Data/new_test/'


### Check OP

In [6]:
import os
solfilename = testroot+'sol.txt'

sol = open(solfilename, 'r')
acans = {}
while True:
    line = sol.readline()
    print(line)
    
    if line == '':
        break
    line = line.split(' ')
    
    acans[line[0]] = line[1]    


0.jpg 03_11.jpg

1.jpg 14_3.jpg

2.jpg 12_8.jpg

3.jpg 02_17.jpg

4.jpg 02_48.jpg

5.jpg 08_12.jpg

6.jpg 08_5.jpg

7.jpg 08_14.jpg

8.jpg 09_25.jpg

9.jpg 09_1.jpg

10.jpg 12_5.jpg

11.jpg 09_9.jpg

12.jpg 02_42.jpg

13.jpg 02_47.jpg

14.jpg 08_14.jpg

15.jpg 08_16.jpg

16.jpg 08_4.jpg

17.jpg 03_4.jpg

18.jpg 03_20.jpg

19.jpg 03_20.jpg

20.jpg 09_32.jpg

21.jpg 02_25.jpg

22.jpg 08_10.jpg

23.jpg 08_4.jpg

24.jpg 14_2.jpg

25.jpg 03_16.jpg

26.jpg 09_28.jpg

27.jpg 03_30.jpg

28.jpg 12_9.jpg

29.jpg 08_13.jpg

30.jpg 09_19.jpg

31.jpg 03_18.jpg

32.jpg 09_9.jpg

33.jpg 09_6.jpg

34.jpg 08_13.jpg

35.jpg 03_13.jpg

36.jpg 09_23.jpg

37.jpg 08_20.jpg

38.jpg 09_27.jpg

39.jpg 14_5.jpg

40.jpg 09_13.jpg

41.jpg 08_12.jpg

42.jpg 14_14.jpg

43.jpg 03_23.jpg

44.jpg 08_2.jpg

45.jpg 14_4.jpg

46.jpg 03_15.jpg

47.jpg 14_7.jpg

48.jpg 12_16.jpg

49.jpg 02_20.jpg

50.jpg 12_5.jpg

51.jpg 08_10.jpg

52.jpg 03_28.jpg

53.jpg 14_24.jpg

54.jpg 03_28.jpg

55.jpg 02_22.jpg

56.jpg 03_15.jpg

57

In [9]:
opfilename = input()
opfilename = testroot+opfilename
op = open(opfilename, 'r')
ac = 0
tot = 0
errors = []
while True:
    line = op.readline()
    
    if line == '':
        break
    
    line = line.split(' ')
    
    if len(line) !=2 :
        print('wrong format')
        break
    
    if acans[line[0]] == line[1]:
        ac+=1
    else:
        a = [line[0], line[1], acans[line[0]]]
        errors.append(a)
        print(a)
    tot+=1

print(ac)
print(ac/tot*100)

op.txt
['6.jpg', '09_1.jpg\n', '08_5.jpg\n']
['7.jpg', '09_8.jpg\n', '08_14.jpg\n']
['20.jpg', '08_20.jpg\n', '09_32.jpg\n']
['93.jpg', '09_14.jpg\n', '09_15.jpg\n']
['102.jpg', '09_9.jpg\n', '08_18.jpg\n']
['86.jpg', '12_15.jpg\n', '12_16.jpg\n']
['89.jpg', '08_20.jpg\n', '09_32.jpg\n']
['75.jpg', '09_4.jpg\n', '08_6.jpg\n']
['73.jpg', '09_1.jpg\n', '08_5.jpg\n']
['54.jpg', '02_9.jpg\n', '03_28.jpg\n']
['48.jpg', '12_15.jpg\n', '12_16.jpg\n']
['41.jpg', '09_8.jpg\n', '08_12.jpg\n']
['182.jpg', '09_9.jpg\n', '08_18.jpg\n']
['170.jpg', '09_8.jpg\n', '08_12.jpg\n']
['158.jpg', '03_9.jpg\n', '03_7.jpg\n']
['141.jpg', '08_2.jpg\n', '09_2.jpg\n']
['122.jpg', '09_9.jpg\n', '08_18.jpg\n']
['112.jpg', '14_16.jpg\n', '03_25.jpg\n']
['113.jpg', '12_15.jpg\n', '12_16.jpg\n']
['114.jpg', '12_15.jpg\n', '12_14.jpg\n']
['250.jpg', '14_16.jpg\n', '03_5.jpg\n']
['242.jpg', '14_0.jpg\n', '03_29.jpg\n']
['232.jpg', '14_0.jpg\n', '09_17.jpg\n']
['198.jpg', '09_8.jpg\n', '08_12.jpg\n']
['305.jpg', '09_8.j